# Групповая сложность бесконечных слов

In [23]:
#список всех подгрупп(full=true) или всех классов сопряженности(full=false)
def get_subgroups(i, full=False):
    if full:
        return SymmetricGroup(i).subgroups()
    return SymmetricGroup(i).conjugacy_classes_subgroups()

In [24]:
#конвертирование строки в массив символов
def convert_to_list(string):
    list = []
    list[:0] = string
    return list

In [25]:
#конвертирование объекта sage в набор позиций, на которые воздействует перестановка
def get_positions(cycle):
    ans = []
    i = 1
    cycle = str(cycle)
    while i < len(cycle) - 1:
        n = ''
        j = 0
        while cycle[i + j] != ',' and cycle[i + j] != ')':
            n += cycle[i + j]
            j += 1
        ans.append(int(n) - 1)
        i += j + 1
        
    return ans

In [26]:
#применение перестановки: перестановки в SAGE записываются в цикленном виде, функция просто проходится
#по каждому циклу из перестановки и ставит на мето текущего элемемнта следующий по циклу
def apply_g(word, g):
    new_word = convert_to_list(word)
    for cycle in g:
        for i in range(len(cycle)):
            new_word[cycle[i]] = word[cycle[(i + 1) % len(cycle)]]
    s = ''.join(new_word)
    return s

In [27]:
#слово Туэ-Морса
def generate_thue():
    s = '01'
    for i in range(1, 100000):
        if s[i] == '1':
            s += '10'
        elif s[i] == '0':
            s += '01'

    return s

#слово period-doubling
def generate_pd():
    s = '01'
    for i in range(1, 1000000):
        if s[i] == '1':
            s += '00'
        elif s[i] == '0':
            s += '01'
    #print(s, file=fout)
    return s

#слово трибоначчи
def generate_trib():
    s = '01'
    for i in range(1, 1000000):
        if s[i] == '1':
            s += '02'
        if s[i] == '0':
            s += '01'
        if s[i] == '2':
            s += '0'
    return s

In [28]:
#подсчет абелевой и факторной сложности и всех факторов, s --- слово, N --- до какой длины считать 
def count_subwords(s, N, show=False, fout=None):
    subwords = []
    complexity = []
    for i in range(4, N):
        c = set()
        ab = set()
        j = 0
        while j + i < len(s):
            w = s[j:j + i]
            c.add(w)
            #ab.add(w.count('0'))
            ab.add((w.count('1'), w.count('2')))
            j += 1

        a = list(c)
        subwords.append(a)
        a.sort()
        complexity.append((len(ab), len(c)))
        if show:
            print(i, ' ab = ', len(ab), 'p = ', len(c), file=fout)
            if i < 10:
                print(*a, sep='\n', file=fout)
    return subwords, complexity

In [47]:
#функция, считающая групповую сложность
def check(words, subgroups):
    res = set()
    num = [0] * (len(words) + 1)
    for i in range(1, len(subgroups)):
        perms = subgroups[i].list()
        classes = {}
        j = 0
        for word in words:
            classes[word] = j
            j += 1
        
        G = []
        for j in range(1, len(perms)):
            g = []
            for cycle in perms[j]:
                g.append(get_positions(cycle))
            G.append(g)
            
        for word in words:
            for g in G:
                w = apply_g(word, g)
                if w in classes.keys():
                    c = classes[w]
                    t = classes[word]
                    if c != t:
                        for k in classes.keys():
                            if classes[k] == c:
                                classes[k] = t
        #print(set(classes.values()))
        #if len(set(classes.values())) not in res:
            #print(len(set(classes.values())), subgroups[i])
        num[len(set(classes.values()))]+=1
        res.add(len(set(classes.values())))
    return res

In [37]:
subwords_trib, complexity_trib = count_subwords(generate_trib(), 20)


Вычисление групповой сложности для слова Трибоначчи. Для 9 и 10 не достигается значение 6, но это может быть связано с тем, что перебор групп не полный. Кроме того, группы, на которых достигаются малые значения, довольно редки

In [51]:
for i in range(5, 11):
    print('n = ', i, 'abelian complexity =', complexity_trib[i-4][0], 'complexity = ', complexity_trib[i-4][1])
    full = False
    if i < 8:
        full = True
    print('group complexity: ', check(set(subwords_trib[i - 4]), get_subgroups(i, full)))

n =  5 abelian complexity = 4 complexity =  11
group complexity:  {4, 5, 6, 7, 8, 9, 10, 11}
n =  6 abelian complexity = 4 complexity =  13
group complexity:  {4, 5, 6, 7, 8, 9, 10, 11, 12, 13}
n =  7 abelian complexity = 4 complexity =  15
group complexity:  {4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15}
n =  8 abelian complexity = 3 complexity =  17
group complexity:  {3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17}
n =  9 abelian complexity = 4 complexity =  19
group complexity:  {4, 5, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19}
n =  10 abelian complexity = 4 complexity =  21
group complexity:  {4, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21}


In [49]:
subwords_pd, complexity_pd = count_subwords(generate_pd(), 20)

Вычисление групповой сложности для period-doubling. Для значения 9 не достигается сложность 4, но скорее всего по тем же причинам, что описаны выше

In [50]:
for i in range(5, 11):
    print('n = ', i, 'abelian complexity =', complexity_pd[i-4][0], 'complexity = ', complexity_pd[i-4][1])
    full = False
    if i < 8:
        full = True
    print('group complexity: ', check(set(subwords_pd[i - 4]), get_subgroups(i, full)))

n =  5 abelian complexity = 3 complexity =  8
group complexity:  {3, 4, 5, 6, 7, 8}
n =  6 abelian complexity = 3 complexity =  10
group complexity:  {3, 4, 5, 6, 7, 8, 9, 10}
n =  7 abelian complexity = 3 complexity =  11
group complexity:  {3, 4, 5, 6, 7, 8, 9, 10, 11}
n =  8 abelian complexity = 2 complexity =  12
group complexity:  {2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12}
n =  9 abelian complexity = 3 complexity =  14
group complexity:  {3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14}
n =  10 abelian complexity = 3 complexity =  16
group complexity:  {3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16}


In [52]:
subwords_thue, complexity_thue = count_subwords(generate_thue(), 20)

Вычисление для Туэ-Морса. На нечетных длинах, как и должно быть, достгаются только четные значения (причем все возможные четные значения).

In [ ]:
for i in range(5, 11):
    print('n = ', i, 'abelian complexity =', complexity_thue[i-4][0], 'complexity = ', complexity_thue[i-4][1])
    full = False
    if i < 7:
        full = True
    print('group complexity: ', check(set(subwords_thue[i - 4]), get_subgroups(i, full)))

n =  5 abelian complexity = 2 complexity =  12
group complexity:  {2, 4, 6, 8, 10}
n =  6 abelian complexity = 3 complexity =  16
group complexity:  {3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14}
n =  7 abelian complexity = 2 complexity =  20
group complexity:  {2, 4, 6, 8, 10, 12, 14, 16, 18, 20}
n =  8 abelian complexity = 3 complexity =  22
group complexity:  {3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 22}
n =  9 abelian complexity = 2 complexity =  24
group complexity:  {2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24}
n =  10 abelian complexity = 3 complexity =  28
